In [ ]:
#pip installations
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install accelerate -U
%pip install datasets
%pip install evaluate
%pip install transformers
%pip install editdistance

## Step -1: Filter Parameters

### Import libraries and get embedding matrix

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

# Load model
tokenizer = AutoTokenizer.from_pretrained("./bert_tokenizer_uncased")
model = AutoModel.from_pretrained("./models/BaseModel_uncased_H100/checkpoint-11380")

# Get full embedding matrix


embedding_matrix = model.get_input_embeddings().weight  # shape: (vocab_size, hidden_size)
print("Embedding matrix shape:", embedding_matrix.shape)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Loading weights: 100%|██████████| 133/133 [00:00<00:00, 574.06it/s, Materializing param=encoder.layer.7.output.dense.weight]              
BertModel LOAD REPORT from: ./models/BaseModel_uncased_H100/checkpoint-11380
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
pooler.dense.bias                          | MISSING    | 
pooler.dense.weight                        | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Embedding matrix shape: torch.Size([30000, 512])


## Step 1: Filter Out Irrelevant Tokens

In [72]:
#first 4981 tokens are unused or single characters
vocab_size = len(tokenizer.get_vocab())
vocab_tokens = [(i,tokenizer.decode([i])) for i in range(4143,vocab_size)]
print(vocab_tokens)

#filter out any tokens with numbers in them
filtered_tokens  = torch.tensor([idx for idx, token in vocab_tokens if not (any((char.isdigit() or char == '#')  for char in token) or len(token) < 3)])
print("Vocab size after removing tokens with numbers:", len(filtered_tokens))
print("Vocab size after removing single char tokens:", len(vocab_tokens))



[(4143, '##0'), (4144, '##7'), (4145, '##e'), (4146, '##t'), (4147, '##r'), (4148, '##o'), (4149, '##b'), (4150, '##i'), (4151, '##l'), (4152, '##a'), (4153, '##y'), (4154, '##n'), (4155, '##z'), (4156, '##s'), (4157, '##c'), (4158, '##h'), (4159, '##v'), (4160, '##1'), (4161, '##4'), (4162, '##3'), (4163, '##2'), (4164, '##u'), (4165, '##k'), (4166, '##p'), (4167, '##w'), (4168, '##g'), (4169, '##d'), (4170, '##f'), (4171, '##x'), (4172, '##m'), (4173, '##j'), (4174, '##ɛ'), (4175, '##ˈ'), (4176, '##ɒ'), (4177, '##θ'), (4178, '##ᵻ'), (4179, '##ʁ'), (4180, '##ɪ'), (4181, '##ʻ'), (4182, '##q'), (4183, '##5'), (4184, '##イ'), (4185, '##ム'), (4186, '##6'), (4187, '##8'), (4188, '##ز'), (4189, '##э'), (4190, '##с'), (4191, '##п'), (4192, '##у'), (4193, '##б'), (4194, '##л'), (4195, '##і'), (4196, '##к'), (4197, '##ʌ'), (4198, '##ː'), (4199, '##τ'), (4200, '##ε'), (4201, '##ν'), (4202, '##ο'), (4203, '##æ'), (4204, '##ø'), (4205, '##ə'), (4206, '##κ'), (4207, '##π'), (4208, '##υ'), (4209, '#

## Step 2: Calculate Similarity Matrix

In [73]:

embeddings = embedding_matrix[filtered_tokens].to(device)

# 1. Normalize embeddings to unit vectors
norm_embeddings = F.normalize(embeddings, p=2, dim=1)

# 2. Calculate the full Similarity Matrix
# Result is (N, N). For 20k tokens, this is ~800MB in float16
sim_matrix = torch.mm(norm_embeddings, norm_embeddings.t())

# 3. Mask the diagonal (Self-similarity is always 1.0)
n = sim_matrix.size(0)
diag_indices = torch.arange(n, device=sim_matrix.device)
sim_matrix[diag_indices, diag_indices] = -1.0  # Set to -1 so topk ignores them

# 4. Get the Top 40 closest neighbors for every single token
k_neighbors = 20
values, indices = torch.topk(sim_matrix, k=k_neighbors, largest=True, dim=1)

# 5. Find the 100 "Closest" pairs globally across the entire matrix
k_global = 10000
flat_values = values.view(-1)
print("Flat values shape:", flat_values.shape)
global_max_vals, global_max_idxs = torch.topk(flat_values, k=k_global * 2, largest=True)

# 6. Map back to token indices
# row_idx: The source word
# neighbor_idx: The similar word
row_indices = global_max_idxs // k_neighbors
neighbor_indices = indices.view(-1)[global_max_idxs]


print("global max vals shape:", global_max_vals.shape)



Flat values shape: torch.Size([384000])
global max vals shape: torch.Size([20000])


In [114]:
theta = 0.55
epsilon = 0

## Filter Pairs Based on Filter Parameters

In [115]:
seen_pairs = set()

for i in range(len(global_max_vals)):
    u, v = row_indices[i].item(), neighbor_indices[i].item()

    word1 = tokenizer.decode([filtered_tokens[u].item()])
    word2 = tokenizer.decode([filtered_tokens[v].item()])
    score = global_max_vals[i].item()
    edit_d_ratio = editdistance.eval(word1, word2) / max(len(word1), len(word2))

    #If the edit distance ratio is greater than epsilon and the score is greater than theta, add to seen pairs
    if edit_d_ratio > epsilon and score >= theta:
        pair = tuple((tuple(sorted((word1, word2))),score))
        
        if pair not in seen_pairs:
            seen_pairs.add(pair)

print("Number of pairs ", len(seen_pairs))
print(seen_pairs)

Number of pairs  4316
{(('feas', 'imposs'), 0.59375), (('cadiz', 'martinique'), 0.5546875), (('implemented', 'instituted'), 0.57421875), (('anything', 'something'), 0.62109375), (('regard', 'respect'), 0.578125), (('decades', 'years'), 0.62109375), (('could', 'might'), 0.66796875), (('consisting', 'consists'), 0.6171875), (('riffs', 'solos'), 0.6328125), (('bermuda', 'newfoundland'), 0.59375), (('croatia', 'slovenia'), 0.578125), (('metac', 'protoc'), 0.57421875), (('guitars', 'synths'), 0.6171875), (('diameter', 'thickness'), 0.59375), (('brings', 'sends'), 0.5625), (('murdered', 'tortured'), 0.55859375), (('destroyers', 'torpedoes'), 0.578125), (('everybody', 'nobody'), 0.55859375), (('ninet', 'twent'), 0.57421875), (('basidia', 'hyphae'), 0.58203125), (('massa', 'raikkonen'), 0.55859375), (('amphibians', 'vertebrates'), 0.55859375), (('fortifications', 'fortresses'), 0.6171875), (('saxophone', 'violin'), 0.57421875), (('assists', 'rebounds'), 0.65625), (('his', 'their'), 0.609375), 

In [116]:

print(f"Number of pairs after edit distance filtering: {len(seen_pairs)}")
print(f"{'Word 1':<20} | {'Word 2':<20} | {'Cosine Sim':<10}")
print("-" * 55)
for pair in seen_pairs:
    word1 = pair[0][0]
    word2 = pair[0][1]
    score = pair[1]

    print(f"{word1:<20} | {word2:<20} | {score:.4f}")

Number of pairs after edit distance filtering: 4316
Word 1               | Word 2               | Cosine Sim
-------------------------------------------------------
feas                 | imposs               | 0.5938
cadiz                | martinique           | 0.5547
implemented          | instituted           | 0.5742
anything             | something            | 0.6211
regard               | respect              | 0.5781
decades              | years                | 0.6211
could                | might                | 0.6680
consisting           | consists             | 0.6172
riffs                | solos                | 0.6328
bermuda              | newfoundland         | 0.5938
croatia              | slovenia             | 0.5781
metac                | protoc               | 0.5742
guitars              | synths               | 0.6172
diameter             | thickness            | 0.5938
brings               | sends                | 0.5625
murdered             | tortured         

## Step 4: Create Counts of Each Token's Occurance and Nearby Tokens

In [117]:
unique_words = dict()
unique_words_related = {}
for pair in seen_pairs:
    unique_words[pair[0][0]] = 0
    unique_words[pair[0][1]] = 0
    unique_words_related[pair[0][0]] = set()
    unique_words_related[pair[0][1]] = set()
print(f"Number of unique words in filtered pairs: {len(unique_words)}")

for pair in seen_pairs:
    unique_words[pair[0][0]] += 1
    unique_words[pair[0][1]] += 1
    unique_words_related[pair[0][0]].add(pair[0][1])
    unique_words_related[pair[0][1]].add(pair[0][0])


sorted_unique_words = sorted(unique_words.items(), key=lambda x: x[1], reverse=True)

Number of unique words in filtered pairs: 4352


In [70]:
print(unique_words_related)

{'battleships': {'destroyers', 'ironclads', 'cruisers', 'dreadnoughts'}, 'cruisers': {'destroyers', 'battleships'}, 'quicker': {'slower'}, 'slower': {'faster', 'quicker'}, 'hadn': {'wouldn', 'weren', 'couldn'}, 'wouldn': {'hadn', 'didn', 'hasn', 'doesn', 'wasn'}, 'decades': {'centuries', 'years'}, 'years': {'days', 'weeks', 'decades', 'months'}, 'could': {'might', 'can'}, 'might': {'would', 'could'}, 'defenceman': {'goaltender'}, 'goaltender': {'defenceman'}, 'riffs': {'solos'}, 'solos': {'riffs'}, 'north': {'east', 'west'}, 'west': {'north', 'south'}, 'british': {'french', 'german'}, 'german': {'british'}, 'guitars': {'drums', 'synths', 'keyboards', 'vocals', 'synthesizers'}, 'synths': {'guitars'}, 'eldest': {'youngest'}, 'youngest': {'eldest'}, 'cockpit': {'fuselage'}, 'fuselage': {'cockpit'}, 'proposed': {'suggested'}, 'suggested': {'proposed'}, 'lower': {'higher', 'upper'}, 'upper': {'lower'}, 'fortifications': {'defences', 'fortresses', 'forts'}, 'fortresses': {'fortifications'}, 

In [82]:
import csv

# Your data: { ( (w1, w2), score ), ... }
data = seen_pairs 

with open("manual_review.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Word 1", "Word 2", "Score"])
    
    # Unpack the nested structure: ((w1, w2), score)
    for (words, score) in data:
        word1, word2 = words
        writer.writerow([word1, word2, score])

print("CSV saved successfully.")

CSV saved successfully.


In [118]:
# print the words withthe highest counts

print(f"{'Word':<20} | {'Count':<10} | {'Related Words'}")
print("-" * 60)
for word, count in sorted_unique_words:
    related_words = ", ".join(sorted(unique_words_related[word]))
    print(f"{word:<20} | {count:<10} | {related_words}")

Word                 | Count      | Related Words
------------------------------------------------------------
twelfth              | 15         | eighteenth, eighth, eleventh, fifteenth, fifth, fourteenth, fourth, ninth, seventeenth, seventh, sixteenth, sixth, tenth, third, thirteenth
fourteenth           | 15         | eighteenth, eighth, eleventh, fifteenth, fifth, nineteenth, ninth, seventeenth, seventh, sixteenth, sixth, tenth, thirteenth, twelfth, twentieth
eight                | 15         | eighteen, eleven, fifteen, five, four, fourteen, nine, seven, six, sixteen, ten, thirteen, three, twelve, two
ninth                | 14         | eighth, eleventh, fifteenth, fifth, fourteenth, fourth, seventeenth, seventh, sixteenth, sixth, tenth, third, thirteenth, twelfth
twelve               | 14         | eight, eighteen, eleven, fifteen, five, four, fourteen, nine, nineteen, seven, six, sixteen, thirteen, three
fifteenth            | 14         | eighteenth, eighth, eleventh, fourteent

## Step 5: Iterate Through Sorted Frequencies and Remove Related Words

In [119]:
#iterate through the most common words and remove all related words

mapping = dict()

removed_words = set()
while len(sorted_unique_words) > 0:
    
    word, count = sorted_unique_words[0]
    if count == 0:
        break
    if word not in unique_words:
        continue

    #Pop the top word from the list
    unique_words.pop(word, None)
    related_words = set(unique_words_related[word])

    # Remove related words from all other entries
    for related_word in related_words:
        mapping[related_word] = word
        removed_words.add(related_word)
        #remove the keys for the related words
        unique_words.pop(related_word, None)
        unique_words_related.pop(related_word, None)
        #remove the instance of the related word from all other related word sets
        for word2 in unique_words:
            if related_word in unique_words_related[word2]:
                unique_words_related[word2].remove(related_word)
                unique_words[word2] -= 1
    sorted_unique_words = sorted(unique_words.items(), key=lambda x: x[1], reverse=True)
    

print(f"Number of removed words: {len(removed_words)}")
print(f"Removed words: {', '.join(sorted(removed_words))}")




Number of removed words: 2631
Removed words: @.@, abc, abdomen, aberdeen, ability, abol, abolished, absurd, academ, accelerate, accolades, accusing, achiev, achievements, acquitted, actin, activ, actor, actresses, adaptations, additionally, additions, adelaide, adequ, adequate, admired, admits, admitting, adriatic, adults, advised, advisers, advisor, affects, affirmed, afraid, afternoon, afterwards, agencies, aggreg, agreed, agreements, ahl, airbus, airways, alabama, alarmed, albania, alberta, albums, aleppo, alerted, allegations, alliances, allmusic, allow, allowed, allowing, allows, almost, already, although, altitudes, aluminium, always, ambival, ambushed, amendments, amidst, amman, amongst, amphibians, amusing, analysts, analyz, anarchist, anatom, android, angeles, anglia, animations, animator, animators, annexed, announced, announces, annoy, annoyed, antananarivo, anterior, anthrop, antib, anticip, antigen, antonio, anxious, anybody, anyone, apologized, appear, appears, applaud, a

In [28]:
print(mapping)

{'ninth': 'twelfth', 'fifteenth': 'twelfth', 'sixth': 'twelfth', 'sixteenth': 'twelfth', 'fourteenth': 'twelfth', 'eleventh': 'twelfth', 'seventh': 'twelfth', 'seventeenth': 'twelfth', 'thirteenth': 'twelfth', 'fifth': 'twelfth', 'january': 'april', 'august': 'april', 'december': 'april', 'march': 'april', 'october': 'april', 'november': 'april', 'september': 'april', 'june': 'april', 'february': 'april', 'july': 'april', 'twelve': 'six', 'eight': 'six', 'nine': 'six', 'five': 'six', 'three': 'six', 'two': 'six', 'seven': 'six', 'four': 'six', 'complimented': 'praised', 'criticized': 'praised', 'appreciated': 'praised', 'lauded': 'praised', 'applauded': 'praised', 'commended': 'praised', 'wouldn': 'didn', 'weren': 'didn', 'doesn': 'didn', 'couldn': 'didn', 'wasn': 'didn', 'hasn': 'didn', 'ninety': 'seventy', 'eighty': 'seventy', 'fifty': 'seventy', 'thirty': 'seventy', 'sixty': 'seventy', 'forty': 'seventy', 'keyboards': 'guitars', 'vocals': 'guitars', 'drums': 'guitars', 'synths': 'gu

## Step 6: Remove Words and Save Tokenizer Vocab

In [120]:
model_state = tokenizer.get_vocab()
for word in removed_words:
    model_state.pop(word, None)

print(model_state)
vocab_list = [token for token, idx in sorted(model_state.items(), key=lambda x: x[1])]
print(vocab_list)
print(len(vocab_list), "words in new vocab")

import json
with open("filtered_tokenizer_vocab_055_0.json", "w", encoding="utf-8") as f:
    json.dump(vocab_list, f, ensure_ascii=False, indent=2)


with open("removed_words_mapping_055_0.json", "w", encoding="utf-8") as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)

{'惑': 2291, '##aghan': 26189, '2003': 7291, 'woodlands': 26846, '##ring': 6769, 'ܢ': 459, 'breakfast': 21448, 'studied': 10673, 'participating': 15648, 'motto': 22392, 'shine': 26071, 'henn': 29863, 'obesity': 28643, '究': 3089, '厩': 1774, 'ref': 6363, '祠': 3052, 'delegate': 21785, 'diverged': 29950, '采': 3742, 'jessica': 18860, 'deacon': 29069, 'honolulu': 28814, 'tiss': 12966, '##ო': 4547, 'expanded': 9714, 'icy': 29746, 'comedies': 29125, '##ව': 4593, 'alexios': 26758, 'volunteer': 15038, 'barric': 21230, '##س': 4264, 'invalid': 23051, 'recovered': 11162, 'advocating': 26648, 'shrubs': 27099, '尧': 2091, '##alus': 19120, 'weld': 15169, 'patches': 20456, '##recht': 24915, 'carved': 16155, 'underw': 12589, '##erted': 10217, 'box': 7763, 'boxes': 18965, 'bolster': 28049, 'skeletons': 27010, '##osed': 5961, 'cds': 26065, 'survival': 13495, 'postseason': 20419, '劳': 1718, 'locked': 17224, '鞍': 3899, 'ლ': 849, 'tig': 13299, 'beneficial': 20857, 'anonymous': 16441, '##ᆺ': 4784, '）': 4088, '淨